In [1]:
import os
os.chdir("../")
%pwd

'/Users/tapankheni/Data_Science/Data Science Projects/Predictive_Maintenance_JetEngine'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from PredictiveMaintenance.constants import CONFIG_YAML_FILE_PATH, SCHEMA_YAML_FILE_PATH, PARAMS_YAML_FILE_PATH
from PredictiveMaintenance.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath=CONFIG_YAML_FILE_PATH,
                 params_filepath=PARAMS_YAML_FILE_PATH, 
                 schema_filepath=SCHEMA_YAML_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [5]:
import os
from urllib import request
import zipfile
from src.PredictiveMaintenance import logger
from src.PredictiveMaintenance.utils.common import get_size

In [6]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-03-06 07:14:57,349: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-06 07:14:57,351: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-06 07:14:57,356: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-06 07:14:57,358: INFO: common: created directory at: artifacts]
[2024-03-06 07:14:57,358: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-06 07:15:04,124: INFO: 2794977711: artifacts/data_ingestion/predictivemaintenance.zip download! with following info 
Connection: close
Content-Length: 12032813
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c472141e2d61999f1d43f2d52ff7118cfd0d8f3ed2b0bcba7e0ff6501ad99a7a"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 473E:40068:1C83F:1F525:65E7CA9A
Accept-Ranges: byt